In [2]:
import random
import nltk
import numpy as np
import json
import pickle

In [59]:
from nltk.stem import WordNetLemmatizer

In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD

In [61]:
lemmatizer = WordNetLemmatizer()

In [3]:
intents = json.loads(open('job_intents.json').read())

In [5]:
#intents

In [63]:
words = []
classes = []
documents = []
ignore_letters = ['?' , '!' , '.' , ',' , ' ? ']

In [64]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list , intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Hey'], 'greeting'), (['Ekse'], 'greeting'), (['Hi'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Ok', 'bye'], 'goodbye'), (['Bye', 'Bye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['What', 'is', 'your', 'name'], 'name'), (['Whats', 'your', 'name'], 'name'), (['Tell', 'me', 'your', 'name'], 'name'), (['Who', 'are', 'you'], 'name'), (['Ungubani'], 'name'), (['Tell', 'me', 'about', 'yourself'], 'name'), (['You', 'are'], 'name'), (['What', 'do', 'you', 'do'], 'options'), (['How', 'can', 'you', 'help'], 'options'), (['What', 'do', 'you', 'know'], 'options'), (['What'

In [65]:
words = [lemmatizer.lemmatize(i) for i in words if i not in ignore_letters]
words = sorted(set(words))

In [66]:
words

["'s",
 'Africa',
 'Awesome',
 'Bye',
 'Ekse',
 'Facts',
 'Give',
 'Good',
 'Goodbye',
 'Hello',
 'Hey',
 'Hi',
 'Hola',
 'How',
 'Is',
 'Ok',
 'SA',
 'See',
 'South',
 'Tell',
 'Thank',
 'Thanks',
 'That',
 'Ungubani',
 'What',
 'Whats',
 'Who',
 'You',
 'about',
 'anyone',
 'are',
 'bye',
 'can',
 'day',
 'do',
 'fact',
 'for',
 'help',
 'helpful',
 'helping',
 'interesting',
 'is',
 'know',
 'later',
 'me',
 'more',
 'name',
 'purpose',
 'some',
 'something',
 'tell',
 'thanks',
 'there',
 'you',
 'your',
 'yourself']

In [67]:
classes = sorted(set(classes))

In [68]:
classes

['goodbye',
 'greeting',
 'name',
 'options',
 'south_africa_facts',
 'south_africa_info',
 'thanks']

In [69]:
pickle.dump(words , open('words.pkl' , 'wb'))
pickle.dump(classes , open('classes.pkl' , 'wb'))

In [70]:
training = []
output_empty = [0]*len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        if word in word_patterns:
            bag.append(1)
        else:
            bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag , output_row])

In [86]:
#l = [[[i,j] for j in range(1,3)] for i in range(1,11)]

In [90]:
#l

In [91]:
#l[0]

In [93]:
training[0]

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0],
 [0, 1, 0, 0, 0, 0, 0]]

In [94]:
random.shuffle(training)
training = np.array(training)

C:\Users\rohan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


In [107]:
train_x = list(training[:,0])
train_y = list(training[:,1])

In [109]:
model = Sequential()
model.add(Dense(64 , input_shape = (len(train_x[0]) , ) ,activation = 'relu')) 
model.add(Dropout(0.5))
model.add(Dense(32 , activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]) , activation = 'softmax'))

sgd = SGD(lr = 0.01 , decay = 1e-6 , momentum=0.9 , nesterov=True)
model.compile(loss = 'categorical_crossentropy' , optimizer = sgd , metrics = ['accuracy'])


In [111]:
model.fit(np.array(train_x) , np.array(train_y) , epochs=50)
model.save('rohan_model.model')
print('Done........!!')

Train on 53 samples
Epoch 1/50
53/53 [==============================] - 2s 29ms/sample - loss: 1.9473 - accuracy: 0.2075
Epoch 2/50
53/53 [==============================] - 0s 468us/sample - loss: 1.9460 - accuracy: 0.1509
Epoch 3/50
53/53 [==============================] - 0s 434us/sample - loss: 1.9528 - accuracy: 0.1698
Epoch 4/50
53/53 [==============================] - 0s 443us/sample - loss: 1.9387 - accuracy: 0.1509
Epoch 5/50
53/53 [==============================] - 0s 418us/sample - loss: 1.8910 - accuracy: 0.1887
Epoch 6/50
53/53 [==============================] - 0s 412us/sample - loss: 1.9204 - accuracy: 0.1698
Epoch 7/50
53/53 [==============================] - 0s 367us/sample - loss: 1.8852 - accuracy: 0.2075
Epoch 8/50
53/53 [==============================] - 0s 470us/sample - loss: 1.9382 - accuracy: 0.1698
Epoch 9/50
53/53 [==============================] - 0s 413us/sample - loss: 1.9064 - accuracy: 0.1887
Epoch 10/50
53/53 [==============================] - 0s 478us/s

In [116]:
np.array(train_x)[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])